# Only required to run from here using sampled_data.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#sampled = np.empty(shape=[0,433])
sampled_csv = './drive/My Drive/ecg/sampled_data.csv'
#with open('./sampled_data.csv', 'rb') as sampled_csv:
# for i in sampled_csv:
#     print('Loading', i)
#     row = np.loadtxt(i, delimiter=',')
#     sampled = np.append(sampled, row, axis=0)
# print(sampled.shape)
print('Loading', sampled_csv)
sampled = np.genfromtxt(sampled_csv, delimiter=',')
print(sampled.shape)

Loading ./drive/My Drive/ecg/sampled_data.csv
(39640, 433)


## Segmenting the data for training and testing using LOOCV Dataset A  
### Intra-patient

In [ ]:
#LOOCV within one patient, choose patient number
# patient = 100
# patient_sample = sampled[sampled[:,-1] == patient]
# print(patient_sample.shape)

In [ ]:
# # LOOCV function to split into train and test for one patient
# # need to figure out average
# def loocv(patient_sample):
#     x = patient_sample[:,:-2]
#     y = patient_sample[:,-2]
#     group = patient_sample[:,-1]

#     cv = LeaveOneOut()
#     n = cv.get_n_splits(x,y=y,groups=group)
#     #print(n)

#     split = cv.split(X=x,y=y,groups=group)

#     train = []
#     test = []
#     for train_index, test_index in split:
#         train.append(train_index)
#         test.append(test_index)

#     train = np.array(train)
#     test = np.array(test)
#     x_train_data = x[train]
#     x_test_data = x[test]
#     y_train_data = y[train]
#     y_test_data = y[test]
    
#     return x_train_data, x_test_data, y_train_data, y_test_data


# x_train_data, x_test_data, y_train_data, y_test_data = loocv(patient_sample)

In [1]:
#Leave one patient out and store each segment in train test arrays
# from sklearn.model_selection import LeaveOneGroupOut
# def leave_one_patient():
#     x = sampled[:,:-2]
#     y = sampled[:,-2]
#     groups = sampled[:,-1]
#     lopo = LeaveOneGroupOut()
#     lopo.get_n_splits(x,y,groups)
#     x_train = []
#     x_test = []
#     y_train = []
#     y_test = []
#     for train_index, test_index in lopo.split(x,y,groups):
#         x_tr, x_te = x[train_index], x[test_index]
#         y_tr, y_te = y[train_index], y[test_index]
#         x_train.append(x_tr)
#         x_test.append(x_te)
#         y_train.append(y_tr)
#         y_test.append(y_te)
#     x_train = np.array(x_train)
#     x_test = np.array(x_test)
#     y_train = np.array(y_train)
#     y_test = np.array(y_test)
#     return x_train, x_test, y_train, y_test

# x_train, x_test, y_train, y_test = leave_one_patient()

In [2]:
# #calculates average for lopo and trains model, returns score
# def score_calculator(model, X_train_arr, y_train_arr, x_test_arr, y_test_arr):
#     score_arr = []
#     for xtr, ytr, xt, yt in zip(X_train_arr, y_train_arr, x_test_arr, y_test_arr):
#         model.fit(xtr, ytr)
#         model_pred = model.predict(xt)
#         score = accuracy_score(yt, model_pred)
#         score_arr.append(score)
    
#     mean_score = np.mean(score_arr)
#     return score_arr, mean_score
    

# 75/25 Train Test Split - Dataset B

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sampled[:,:-3], sampled[:,-3], test_size=0.25, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

NameError: name 'sampled' is not defined

#### Confusion Matrix Function

In [ ]:
def scores(y_test, predictions):
  accuracy = accuracy_score(y_test, predictions)
  precision_micro = precision_score(y_test, predictions, average='micro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
  precision_macro = precision_score(y_test, predictions, average='macro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
  f1_macro = f1_score(y_test, predictions, average='macro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
  f1_micro = f1_score(y_test, predictions, average='micro', labels=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0,8.0])
  return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [ ]:
def cf_matrix_func(y_test, predictions):
    cm = confusion_matrix(y_test, predictions)
    categories=['N', 'L', 'R', 'A', 'V', 'F', 'f','P']
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    sns.heatmap(cm, annot=True, 
                fmt='.2%', cmap='Blues',xticklabels=categories,yticklabels=categories)    

## KERAS CNN

In [ ]:
from keras.utils.np_utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [ ]:
X_train_3d = X_train.reshape(-1, X_train.shape[1],1)
X_test_3d = X_test.reshape(-1, X_test.shape[1], 1)

print(X_train_3d.shape)
print(X_test_3d.shape)

(29730, 430, 1)
(9910, 430, 1)


In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
print(y_train_cat.shape)
print(y_test_cat.shape)

(29730, 9)
(9910, 9)


In [ ]:
verbose, epoch, batch_size = 1, 100, 32

def getModel():
    cnnmodel = Sequential()
    cnnmodel.add(Conv1D(filters=128, kernel_size=2, activation='relu',input_shape=(X_train_3d.shape[1],X_train_3d.shape[2])))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
    cnnmodel.add(MaxPooling1D(pool_size=2))
    cnnmodel.add(Dropout(rate=0.2))
    cnnmodel.add(Flatten())
    cnnmodel.add(Dense(64, activation='relu'))
    cnnmodel.add(Dense(32, activation='relu'))
    cnnmodel.add(Dense(9, activation='softmax'))
    cnnmodel.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    cnnmodel.summary()
    return cnnmodel

cnnmodel = getModel()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 429, 128)          384       
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 214, 128)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 213, 64)           16448     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 106, 64)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 105, 32)           4128      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 52, 32)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 51, 16)           

In [ ]:
model = cnnmodel.fit(X_train_3d, y_train_cat, epochs=epoch, verbose=verbose,batch_size=batch_size)

Epoch 1/100
930/930 [==============================] - 46s 49ms/step - loss: 0.4977 - accuracy: 0.8349
Epoch 2/100
930/930 [==============================] - 46s 50ms/step - loss: 0.3371 - accuracy: 0.8893
Epoch 3/100
930/930 [==============================] - 47s 50ms/step - loss: 0.2711 - accuracy: 0.9090
Epoch 4/100
930/930 [==============================] - 47s 50ms/step - loss: 0.2364 - accuracy: 0.9197
Epoch 5/100
930/930 [==============================] - 47s 51ms/step - loss: 0.2064 - accuracy: 0.9316
Epoch 6/100
930/930 [==============================] - 47s 51ms/step - loss: 0.1882 - accuracy: 0.9358
Epoch 7/100
930/930 [==============================] - 47s 51ms/step - loss: 0.1733 - accuracy: 0.9411
Epoch 8/100
930/930 [==============================] - 47s 50ms/step - loss: 0.1551 - accuracy: 0.9466
Epoch 9/100
930/930 [==============================] - 47s 50ms/step - loss: 0.1468 - accuracy: 0.9499
Epoch 10/100
930/930 [==============================] - 46s 49ms/step - l

In [ ]:
cnnpredictions = cnnmodel.predict(X_test_3d, verbose=1)

310/310 [==============================] - 5s 17ms/step


In [ ]:
#print(cnnpredictions)

cnn_predict = np.argmax(cnnpredictions, axis=1)
true_value = np.argmax(y_test_cat,axis=1)

In [ ]:
from sklearn.metrics import *
score = scores(true_value, cnn_predict)
print(score)

(0.981029263370333, 0.9809929172462879, 0.981029263370333, 0.9809000038753486, 0.981029263370333)


### Saving the Model

In [ ]:
cnnmodel.save('./drive/My Drive/ecg/cnnmodel.h5')
cnnmodel.save_weights('./drive/My Drive/ecg/cnnmodelweights.h5')

In [ ]:
model_json = cnnmodel.to_json()
with open ("./drive/My Drive/ecg/cnnmodel.json", "w") as jf:
  jf.write(model_json)

### Reading the model

In [ ]:
from keras.models import model_from_json

jf = open("./drive/My Drive/ecg/cnnmodel.json", "r")
loaded_model = jf.read()
jf.close()

lm = model_from_json(loaded_model)

lm.load_weights("./drive/My Drive/ecg/cnnmodelweights.h5")

lm.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

score_cnn = scores(true_value, cnn_predict)
print(score_cnn)


(0.981029263370333, 0.9809929172462879, 0.981029263370333, 0.9809000038753486, 0.981029263370333)


# Resnet